<a href="https://colab.research.google.com/github/dimasrfaria/falvojr-santander-dev-week-2023/blob/master/santanderDevWeek_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('SDW2026.csv', sep=';')

# Exibe informações sobre os dados (tipos e uso de memória)
df.info()

print()  # Linha em branco para melhor visualização

# Exibe todas as informações em formato de tabela
print(df.to_string(index=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UserID  5 non-null      int64 
 1   Nome    5 non-null      object
 2   Conta   5 non-null      object
 3   Cartao  5 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 292.0+ bytes

 UserID     Nome  Conta  Cartao
      1     João 0001-1   90001
      2   Felipe 0002-2   90002
      3   Maduro 0003-3   90003
      4    Maria 0004-4   90004
      5 Josefina 0005-5   90005


In [2]:
# -----------------------------
# 2. Função de geração da News
# (stub - será substituída pela API futuramente)
# -----------------------------

import os
from datetime import datetime

def gerar_news(usuario):
    return (
        f"Olá {usuario['Nome']}, temos soluções financeiras pensadas especialmente "
        f"para apoiar seus objetivos e oferecer mais tranquilidade no dia a dia."
    )

# -----------------------------
# 3. Geração das mensagens (News)
# -----------------------------
news_geradas = []

data_geracao = datetime.now().strftime("%d-%m-%y %H:%M:%S")

for _, usuario in df.iterrows():
    news_geradas.append({
        "UserID": usuario["UserID"],
        "Nome": usuario["Nome"],
        "News": gerar_news(usuario),
        "DataGeracao": data_geracao
    })

df_news = pd.DataFrame(news_geradas)



In [3]:
# -----------------------------
# 4. Escrita no CSV com histórico
# -----------------------------
arquivo_saida = "SDW2026_news.csv"
arquivo_existe = os.path.exists(arquivo_saida)

df_news.to_csv(
    arquivo_saida,
    sep=";",
    index=False,
    encoding="utf-8",
    mode="a",                 # append (mantém histórico)
    header=not arquivo_existe # escreve cabeçalho só se o arquivo não existir
)